# LAB | Extractive Question Answering

This notebook demonstrates how Pinecone helps you build an extractive question-answering application. To build an extractive question-answering system, we need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A reader model to extract answers

We will use the SQuAD dataset, which consists of **questions** and **context** paragraphs containing question **answers**. We generate embeddings for the context passages using the retriever, index them in the vector database, and query with semantic search to retrieve the top k most relevant contexts containing potential answers to our question. We then use the reader model to extract the answers from the returned contexts.

Let's get started by installing the packages needed for notebook to run:

In [45]:
!pip install -qU python-dotenv datasets pinecone-client sentence-transformers torch langchain-pinecone pinecone-notebooks

In [46]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY= os.getenv('PINECONE_API_KEY')

In [47]:
print("PINECONE_API_KEY :", bool(os.getenv("PINECONE_API_KEY")))
print("OPENAI_API_KEY :", bool(os.getenv("OPENAI_API_KEY")))


PINECONE_API_KEY : True
OPENAI_API_KEY : True


# Install Dependencies

# Load Dataset

Now let's load the SQUAD dataset from the HuggingFace Model Hub. We load the dataset into a pandas dataframe and filter the title, question, and context columns, and we drop any duplicate context passages.

In [48]:
from datasets import load_dataset

# load the squad dataset into a pandas dataframe
df = load_dataset("squad", split="train").to_pandas()

In [49]:
df.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


In [50]:
# select only title and context column
df = df[['title', 'context']]
# drop rows containing duplicate context passages
df = df.drop_duplicates(subset=['context'])
df.head()

,title,context
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha..."
5,University_of_Notre_Dame,"As at most other universities, Notre Dame's st..."
10,University_of_Notre_Dame,The university is the major seat of the Congre...
15,University_of_Notre_Dame,The College of Engineering was established in ...
20,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our context passages which we can retrieve using another vector (query vector). We first need to initialize our connection to Pinecone to create our vector index. For this, we need a free [API key]("https://app.pinecone.io/"), and then we initialize the connection like so:

In [51]:
from pinecone import Pinecone, ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

# connect to pinecone environment
pc = Pinecone(
    api_key = PINECONE_API_KEY,
    environment='us-east-1'  # find next to API key in console
)

Now we create a new index called "question-answering" — we can name the index anything we want. We specify the metric type as "cosine" and dimension as 384 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 384-dimension vectors.

In [52]:
index_name = "question-answering"

# check if the index exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=spec
    )
index = pc.Index(index_name)

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all context passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will generate embeddings in a way that the questions and context passages containing answers to our questions are nearby in the vector space. We can use cosine similarity to calculate the similarity between the query and context embeddings to find the context passages that contain potential answers to our question.

We will use a SentenceTransformer model named ``multi-qa-MiniLM-L6-cos-v1`` designed for semantic search and trained on 215M (question, answer) pairs from diverse sources as our retriever.

In [53]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1').to(device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, context passage, etc.

In [54]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    i_end = min(i+batch_size, len(df))
    batch = df.iloc[i:i_end]

    # generate embeddings
    emb = retriever.encode(batch["context"].tolist()).tolist()

    # prepare metadata
    meta = [{"title": title, "context": context} for title, context in zip(batch["title"], batch["context"])]

    # create unique IDs
    ids = [f"{i+j}" for j in range(len(batch))]

    # prepare list of tuples (id, vector, metadata)
    to_upsert = list(zip(ids, emb, meta))

    # upsert to Pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/296 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 18891}},
 'total_vector_count': 18891,
 'vector_type': 'dense'}

# Initialize Reader

We use the `deepset/electra-base-squad2` model from the HuggingFace model hub as our reader model. We load this model into a "question-answering" pipeline from HuggingFace transformers and feed it our questions and context passages individually. The model gives a prediction for each context we pass through the pipeline.

In [55]:
from transformers import pipeline

model_name = 'deepset/electra-base-squad2'
# load the reader model into a question-answering pipeline
reader = pipeline(tokenizer=model_name, model=model_name, task='question-answering', device=device)
reader

Device set to use cuda


Now all the components we need are ready. Let's write some helper functions to execute our queries. The `get_context` function retrieves the context embeddings containing answers to our question from the Pinecone index, and the `extract_answer` function extracts the answers from these context passages.

In [56]:
# gets context passages from the pinecone index
def get_context(question, top_k=3):
    # create query vector
    xq = retriever.encode([question]).tolist()

    # search Pinecone
    xc = index.query(vector=xq[0], top_k=top_k, include_metadata=True)

    # extract context passages
    c = [match['metadata']['context'] for match in xc['matches']]
    return c

In [57]:
from pprint import pprint

# extracts answer from the context passage
def extract_answer(question, context):
    results = []
    for c in context:
        # feed the reader the question and contexts to extract answers
        answer = reader(question=question, context=c)
        # add the context to answer dict for printing both together
        answer["context"] = c
        results.append(answer)
    # sort the result based on the score from reader model
    sorted_result = pprint(sorted(results, key=lambda x: x['score'], reverse=True))
    return sorted_result

In [58]:
question = "How much oil is Egypt producing in a day?"
context = get_context(question, top_k = 1)
context

['Egypt was producing 691,000 bbl/d of oil and 2,141.05 Tcf of natural gas (in 2013), which makes Egypt as the largest oil producer not member of the Organization of the Petroleum Exporting Countries (OPEC) and the second-largest dry natural gas producer in Africa. In 2013, Egypt was the largest consumer of oil and natural gas in Africa, as more than 20% of total oil consumption and more than 40% of total dry natural gas consumption in Africa. Also, Egypt possesses the largest oil refinery capacity in Africa 726,000 bbl/d (in 2012). Egypt is currently planning to build its first nuclear power plant in El Dabaa city, northern Egypt.']

As we can see, the retiever is working fine and gets us the context passage that contains the answer to our question. Now let's use the reader to extract the exact answer from the context passage.

In [59]:
extract_answer(question, context)

[{'answer': '691,000 bbl/d',
  'context': 'Egypt was producing 691,000 bbl/d of oil and 2,141.05 Tcf of '
             'natural gas (in 2013), which makes Egypt as the largest oil '
             'producer not member of the Organization of the Petroleum '
             'Exporting Countries (OPEC) and the second-largest dry natural '
             'gas producer in Africa. In 2013, Egypt was the largest consumer '
             'of oil and natural gas in Africa, as more than 20% of total oil '
             'consumption and more than 40% of total dry natural gas '
             'consumption in Africa. Also, Egypt possesses the largest oil '
             'refinery capacity in Africa 726,000 bbl/d (in 2012). Egypt is '
             'currently planning to build its first nuclear power plant in El '
             'Dabaa city, northern Egypt.',
  'end': 33,
  'score': 0.9999852180480957,
  'start': 20}]


The reader model predicted with 99% accuracy the correct answer *691,000 bbl/d* as seen from the context passage. Let's run few more queries.

In [60]:
question = "What are the first names of the men that invented youtube?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'Hurley and Chen',
  'context': 'According to a story that has often been repeated in the media, '
             'Hurley and Chen developed the idea for YouTube during the early '
             'months of 2005, after they had experienced difficulty sharing '
             "videos that had been shot at a dinner party at Chen's apartment "
             'in San Francisco. Karim did not attend the party and denied that '
             'it had occurred, but Chen commented that the idea that YouTube '
             'was founded after a dinner party "was probably very strengthened '
             'by marketing ideas around creating a story that was very '
             'digestible".',
  'end': 79,
  'score': 0.9999276399612427,
  'start': 64}]


In [61]:
question = "What is Albert Eistein famous for?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'his theories of special relativity and general relativity',
  'context': 'Albert Einstein is known for his theories of special relativity '
             'and general relativity. He also made important contributions to '
             'statistical mechanics, especially his mathematical treatment of '
             'Brownian motion, his resolution of the paradox of specific '
             'heats, and his connection of fluctuations and dissipation. '
             'Despite his reservations about its interpretation, Einstein also '
             'made contributions to quantum mechanics and, indirectly, quantum '
             'field theory, primarily through his theoretical studies of the '
             'photon.',
  'end': 86,
  'score': 0.9500371217727661,
  'start': 29}]


Let's run another question. This time for top 3 context passages from the retriever.

In [62]:
question = "Who was the first person to step foot on the moon?"
context = get_context(question, top_k=3)
extract_answer(question, context)

[{'answer': 'Armstrong',
  'context': 'The trip to the Moon took just over three days. After achieving '
             'orbit, Armstrong and Aldrin transferred into the Lunar Module, '
             'named Eagle, and after a landing gear inspection by Collins '
             'remaining in the Command/Service Module Columbia, began their '
             'descent. After overcoming several computer overload alarms '
             'caused by an antenna switch left in the wrong position, and a '
             'slight downrange error, Armstrong took over manual flight '
             'control at about 180 meters (590 ft), and guided the Lunar '
             'Module to a safe landing spot at 20:18:04 UTC, July 20, 1969 '
             '(3:17:04 pm CDT). The first humans on the Moon would wait '
             'another six hours before they ventured out of their craft. At '
             '02:56 UTC, July 21 (9:56 pm CDT July 20), Armstrong became the '
             'first human to set foot on the Moon.',

The result looks pretty good.

In [63]:
#pc.delete_index(index_name)

### Add a few more questions. What did you observe?

In [72]:
question = "Who built the Eiffel Tower?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'Thomas Cubitt',
  'context': 'By 1847, the couple had found the palace too small for court '
             'life and their growing family, and consequently the new wing, '
             'designed by Edward Blore, was built by Thomas Cubitt, enclosing '
             'the central quadrangle. The large East Front, facing The Mall, '
             'is today the "public face" of Buckingham Palace, and contains '
             'the balcony from which the royal family acknowledge the crowds '
             'on momentous occasions and after the annual Trooping the Colour. '
             'The ballroom wing and a further suite of state rooms were also '
             "built in this period, designed by Nash's student Sir James "
             'Pennethorne.',
  'end': 175,
  'score': 0.5605766177177429,
  'start': 162}]


In [73]:
question = "What is Python programming language used for?"
context = get_context(question, top_k=2)
extract_answer(question, context)

[{'answer': 'specifying programs for computers to run',
  'context': 'Programming languages provide various ways of specifying '
             'programs for computers to run. Unlike natural languages, '
             'programming languages are designed to permit no ambiguity and to '
             'be concise. They are purely written languages and are often '
             'difficult to read aloud. They are generally either translated '
             'into machine code by a compiler or an assembler before being '
             'run, or translated directly at run time by an interpreter. '
             'Sometimes programs are executed by a hybrid method of the two '
             'techniques.',
  'end': 86,
  'score': 0.0010914959711953998,
  'start': 46},
 {'answer': 'write computer programs as long lists of numbers',
  'context': 'While it is possible to write computer programs as long lists of '
             'numbers (machine language) and while this technique was used '
             'with m

In [74]:
question = "Who is the president of the University of Notre Dame?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'John I. Jenkins',
  'context': 'Since 2005, Notre Dame has been led by John I. Jenkins, C.S.C., '
             'the 17th president of the university. Jenkins took over the '
             'position from Malloy on July 1, 2005. In his inaugural address, '
             'Jenkins described his goals of making the university a leader in '
             'research that recognizes ethics and building the connection '
             'between faith and studies. During his tenure, Notre Dame has '
             'increased its endowment, enlarged its student body, and '
             'undergone many construction projects on campus, including '
             'Compton Family Ice Arena, a new architecture hall, additional '
             'residence halls, and the Campus Crossroads, a $400m enhancement '
             'and expansion of Notre Dame Stadium.',
  'end': 54,
  'score': 0.9935050010681152,
  'start': 39}]


In [75]:
question = "What does DNA stand for?"
context = get_context(question, top_k=3)
extract_answer(question, context)

[{'answer': 'Deoxyribonucleic acid',
  'context': 'Advances in understanding genes and inheritance continued '
             'throughout the 20th century. Deoxyribonucleic acid (DNA) was '
             'shown to be the molecular repository of genetic information by '
             'experiments in the 1940s to 1950s. The structure of DNA was '
             'studied by Rosalind Franklin using X-ray crystallography, which '
             'led James D. Watson and Francis Crick to publish a model of the '
             'double-stranded DNA molecule whose paired nucleotide bases '
             'indicated a compelling hypothesis for the mechanism of genetic '
             'replication. Collectively, this body of research established the '
             'central dogma of molecular biology, which states that proteins '
             'are translated from RNA, which is transcribed from DNA. This '
             'dogma has since been shown to have exceptions, such as reverse '
             'transcription

In [76]:
question = "When did World War II end?"
context = get_context(question, top_k=2)
extract_answer(question, context)


[{'answer': 'Cold War',
  'context': 'The end of World War II set the stage for the East–West '
             'confrontation known as the Cold War. With the outbreak of the '
             'Korean War, concerns over the defense of Western Europe rose. '
             'Two corps, V and VII, were reactivated under Seventh United '
             'States Army in 1950 and American strength in Europe rose from '
             'one division to four. Hundreds of thousands of U.S. troops '
             'remained stationed in West Germany, with others in Belgium, the '
             'Netherlands and the United Kingdom, until the 1990s in '
             'anticipation of a possible Soviet attack.',
  'end': 91,
  'score': 0.0003886170161422342,
  'start': 83},
 {'answer': '1918',
  'context': 'The outbreak of World War I in 1914 was precipitated by the rise '
             'of nationalism in Southeastern Europe as the Great Powers took '
             'up sides. The Allies defeated the Central Powers in 1

In [77]:
question = "What is the Grotto at Notre Dame?"
context = get_context(question, top_k=1)
extract_answer(question, context)

[{'answer': 'a Marian place of prayer and reflection',
  'context': 'Architecturally, the school has a Catholic character. Atop the '
             "Main Building's gold dome is a golden statue of the Virgin Mary. "
             'Immediately in front of the Main Building and facing it, is a '
             'copper statue of Christ with arms upraised with the legend '
             '"Venite Ad Me Omnes". Next to the Main Building is the Basilica '
             'of the Sacred Heart. Immediately behind the basilica is the '
             'Grotto, a Marian place of prayer and reflection. It is a replica '
             'of the grotto at Lourdes, France where the Virgin Mary reputedly '
             'appeared to Saint Bernadette Soubirous in 1858. At the end of '
             'the main drive (and in a direct line that connects through 3 '
             'statues and the Gold Dome), is a simple, modern stone statue of '
             'Mary.',
  'end': 420,
  'score': 0.9997685551643372,
  'start': 3


## **🔍 Observations:**

### The retriever (SentenceTransformer) often returns irrelevant context when the question is not aligned with the dataset content (e.g., Eiffel Tower → Buckingham Palace).

### Fact-based questions (e.g., "What does DNA stand for?") yield high-confidence, accurate answers, especially when relevant context is found.

### Increasing top_k improves the system’s ability to find correct answers by searching a broader range of contexts.

### Answer quality heavily depends on get_context() returning relevant text — otherwise, the reader may output incorrect or misleading answers.

### Best results occur when the questions are clearly worded and closely related to the document/domain.
